In [1]:
import numpy as np

In [2]:
def process_gro_file(gro_file, atoms):
    with open(gro_file, 'r') as file:
        lines = file.readlines()

    # Skip the first two lines (title and number of atoms)
    atom_lines = lines[2:-1]

    # Dictionary to map residue numbers to molecule IDs
    residue_to_molecule_id = {}
    molecule_id_counter = 1

    for line in atom_lines:
        # Using split() to extract atom information from each line
        parts = line.split()

        if len(parts) >= 6:
            # Extracting atom information
            residue_number = parts[0]
            atom_name = parts[1]
            atom_id = int(parts[2])
            x, y, z = float(parts[3]), float(parts[4]), float(parts[5])

            # Convert coordinates from nm to Angstroms (1 nm = 10 Angstroms)
            x, y, z = [coord * 10 for coord in [x, y, z]]

            # Assign a molecule ID to each unique residue number
            if residue_number not in residue_to_molecule_id:
                residue_to_molecule_id[residue_number] = molecule_id_counter
                molecule_id_counter += 1

            molecule_id = residue_to_molecule_id[residue_number]

            # Append the atom's information to the 'atoms' list
            atom_data = {
                'atom_id': atom_id,
                'molecule_id': molecule_id,
                'atom_name': atom_name,
                'residue_number': residue_number,
                'x': x,
                'y': y,
                'z': z,
                'atom_type':'UNKOWN',
                'charge':'UNKOWN'
            }
            atoms.append(atom_data)

In [3]:
def process_itp_file(itp_file, atoms, bonds, angles, dihedrals, impropers, pairs, i=0):
    with open(itp_file, 'r') as file:
        lines = file.readlines()

    section = None
    for line in lines:
        if line.strip() =="; Include Position restraint file":
            break

        if line.startswith(';') or not line.strip():
            continue

        if line.startswith('['):
            section = line.strip().strip('[]').strip()
            continue

        if section == 'atoms':
            process_atoms_line(line, atoms, i)
            atom_id_to_type = {atom['atom_id']: atom['atom_type'] for atom in atoms}
        elif section == 'bonds':
            process_bonds_line(line, bonds)
        elif section == 'pairs':
            process_unique_pairs_line(line, pairs, atom_id_to_type)
        elif section == 'angles':
            process_angles_line(line, angles)
        elif section == 'dihedrals':
            process_dihedrals_line(line, dihedrals, impropers)


def process_atoms_line(line, atoms, i):
    # Assuming atom_id in atoms list corresponds to atom numbering in .itp file
    parts = line.split()
    atom_id = int(parts[0])+i*188
    atom_types = {'NT': 1, 'H': 2, 'CH1': 3, 'CH2':4, 'C':5,'O':6,'N':7,'OM':8}  # Add other atom types as needed
    
    atom_type_str = parts[1]
    if atom_type_str in atom_types:
        atom_type = str(atom_types[atom_type_str])
    else:
        print(atom_type_str)

    charge = float(parts[6])
    # Update atom type and charge for corresponding atom
    for atom in atoms:
        if atom['atom_id'] == atom_id:
            atom['atom_type'] = atom_type
            atom['charge'] = charge
            break

def process_unique_pairs_line(line, pairs, atom_id_to_type):
    parts = line.split()

    if len(parts) < 3:
        return  # Skip lines that don't have enough data

    atom_type1 = atom_id_to_type[int(parts[0])]
    atom_type2 = atom_id_to_type[int(parts[1])]

    # Create a pair key, ensuring consistent ordering for uniqueness
    pair_key = tuple(sorted([atom_type1, atom_type2]))

    # Add the pair key to the set of unique pairs
    pairs.append(pair_key)
    
def process_bonds_line(line, bonds):
    bond_types = {'gb_2': 1, 'gb_20': 2, 'gb_26': 3, 'gb_4': 4, 'gb_9': 5,'gb_5':6}

    # https://github.com/gromacs/gromacs/blob/main/share/top/gromos43a1.ff/ffbonded.itp
    # force_field
    parts = line.split()

    # Check if the bond type is in the bond_types dictionary
    bond_type_str = parts[3]  # The bond type string is in the fourth column
    if bond_type_str in bond_types:
        bond_type = bond_types[bond_type_str]

        bond = {
            'bond_type': bond_type,
            'atom_id1': int(parts[0]),
            'atom_id2': int(parts[1])
        }
        
        bonds.append(bond)

    return bonds

def process_angles_line(line, angles):
    parts = line.split()
    angle_types = {'ga_9': 1, 'ga_10': 2, 'ga_20': 3, 'ga_12': 4, 'ga_29': 5,'ga_18':6,'ga_32':7,
                   'ga_30':8}

    # Check if the bond type is in the bond_types dictionary
    angle_type_str = parts[4]  # The bond type string is in the fourth column
    if angle_type_str in angle_types:
        angle_type = angle_types[angle_type_str]

        angle = {
            'angle_type': angle_type,
            'atom_id1': int(parts[0]),
            'atom_id2': int(parts[1]),
            'atom_id3': int(parts[2])
        }

        angles.append(angle)

def process_dihedrals_line(line, dihedrals, impropers):
    parts = line.split()
    dihedral_types = {'gd_19': 1, 'gd_17': 2, 'gd_20': 3, 'ga_4': 4}
    improper_types={'gi_1':1,'gi_2':2}
    # Check if the bond type is in the bond_types dictionary
    dihedral_type_str = parts[5]  # The bond type string is in the fourth column
    if dihedral_type_str in dihedral_types:
        dihedral_type = dihedral_types[dihedral_type_str]

        dihedral = {
            'dihedral_type': dihedral_type,
            'atom_id1': int(parts[0]),
            'atom_id2': int(parts[1]),
            'atom_id3': int(parts[2]),
            'atom_id4': int(parts[3])
        }

        dihedrals.append(dihedral)
    
    if dihedral_type_str in improper_types:
        dihedral_type = improper_types[dihedral_type_str]

        improper = {
            'dihedral_type': dihedral_type,
            'atom_id1': int(parts[0]),
            'atom_id2': int(parts[1]),
            'atom_id3': int(parts[2]),
            'atom_id4': int(parts[3])
        }

        impropers.append(improper)
        


In [4]:
def remove_duplicate_pairs(pairs_list):
    unique_pairs = set()
    for pair in pairs_list:
        sorted_pair = tuple(sorted(pair))
        unique_pairs.add(sorted_pair)

    return list(unique_pairs)


def convert_numbers_to_atom_types(pairs_list, atom_types):
    # Invert the atom_types dictionary to map numbers to atom type names
    number_to_atom_type = {v: k for k, v in atom_types.items()}

    converted_pairs = []
    for pair in pairs_list:
        atom_type1 = number_to_atom_type.get(int(pair[0]), "Unknown")
        atom_type2 = number_to_atom_type.get(int(pair[1]), "Unknown")
        converted_pairs.append((atom_type1, atom_type2))

    return converted_pairs

In [5]:
def write_lammps_data_file(data_file, atoms, bonds, angles, dihedrals, impropers, pairs):
    with open(data_file, 'w') as file:
        # Write header
        file.write("LAMMPS Description\n\n")
        file.write(f"{len(atoms)} atoms\n")
        # file.write(f"{len(pairs_list)} pairs\n")
        file.write(f"{len(bonds)} bonds\n")
        file.write(f"{len(angles)} angles\n")
        file.write(f"{len(dihedrals)} dihedrals\n")
        file.write(f"{len(impropers)} impropers\n\n")


        atom_types = {'NT': 1, 'H': 2, 'CH1': 3, 'CH2':4, 'C':5,'O':6,'N':7,'OM':8}  # Add other atom types as needed


        bond_types = {'gb_2': 1, 'gb_20': 2, 'gb_26': 3, 'gb_4': 4, 'gb_9': 5,'gb_5':6}

        angle_types = {'ga_9': 1, 'ga_10': 2, 'ga_20': 3, 'ga_12': 4, 'ga_29': 5,'ga_18':6,'ga_32':7,'ga_30':8}
        dihedral_types_1 = {'gd_19': 1, 'gd_17': 2, 'gd_20': 3, 'ga_4': 4}
        dihedral_types_2={'gi_1':1,'gi_2':2}
        file.write(f"{len(atom_types)} atom types\n")  
        file.write(f"{len(bond_types)} bond types\n")
        file.write(f"{len(angle_types)} angle types\n")
        file.write(f"{len(dihedral_types_1)} dihedral types\n")
        file.write(f"{len(dihedral_types_2)} improper types\n\n")

        # Box dimensions (to be determined from your system)

        # x, y, z max 17.15 26.79 180.97
        # x, y, z min 6.06 16.03 1.13
        
        file.write("0.0 40.0 xlo xhi\n")
        file.write("0.0 40.0 ylo yhi\n")
        file.write("0.0 200.0 zlo zhi\n\n")

        file.write("Masses\n\n")
        atom_types = {'NT': 1, 'H': 2, 'CH1': 3, 'CH2':4, 'C':5,'O':6,'N':7,'OM':8} 
        file.write("1 14.0067  \n"
                   "2 1.008 \n"
                   "3 13.019 \n"
                   "4 14.027 \n"
                   "5 12.0122 \n"
                   "6 16.994 \n"
                   "7 14.0067 \n"
                   "8 15.9994 \n\n")  

        # Atoms section
        file.write("Atoms\n\n")
        for atom in atoms:
            file.write(f"{atom['atom_id']} {atom['molecule_id']} {atom['atom_type']} {atom['charge']} "
                       f"{atom['x']} {atom['y']} {atom['z']}\n")
     
        # Bonds section 
    
        file.write("\nBonds\n\n")
        for i, bond in enumerate(bonds, start=1):
            file.write(f"{i} {bond['bond_type']} {bond['atom_id1']} {bond['atom_id2']}\n")

        file.write("\nAngles\n\n")
        for i, angle in enumerate(angles, start=1):
            file.write(f"{i} {angle['angle_type']} {angle['atom_id1']} "
                        f"{angle['atom_id2']} {angle['atom_id3']}\n")

        file.write("\nDihedrals\n\n")
        for i, dihedral in enumerate(dihedrals, start=1):
            file.write(f"{i} {dihedral['dihedral_type']} {dihedral['atom_id1']} "
                        f"{dihedral['atom_id2']} {dihedral['atom_id3']} {dihedral['atom_id4']}\n")

        file.write("\nImpropers\n\n")
        for i, improper in enumerate(impropers, start=1):
            file.write(f"{i} {improper['dihedral_type']} {improper['atom_id1']} "
                        f"{improper['atom_id2']} {improper['atom_id3']} {improper['atom_id4']}\n")

In [6]:
def convert_to_lammps_data(gro_file, itp_files, lammps_data_file):
    atoms, bonds, angles, dihedrals,impropers, pairs  = [], [], [], [], [], []
    # Process .gro and .itp files
    process_gro_file(gro_file, atoms)
    i=0
    for itp_file in itp_files:
        process_itp_file(itp_file, atoms, bonds, angles, dihedrals, impropers, pairs, i)
        i+=1

    atom_types = {'NT': 1, 'H': 2, 'CH1': 3, 'CH2':4, 'C':5,'O':6,'N':7,'OM':8}  # Add other atom types as needed
    
    pairs=remove_duplicate_pairs(pairs)
    converted_pairs = convert_numbers_to_atom_types(pairs, atom_types)
    print(converted_pairs)
    print(pairs)

    # Write to LAMMPS data file
    write_lammps_data_file(lammps_data_file, atoms, bonds, angles, dihedrals, impropers, pairs)

gro_file = '../pdb_gromacs/output.gro'
itp_files = ['../pdb_gromacs/topol_Protein_chain_A.itp', '../pdb_gromacs/topol_Protein_chain_B.itp','../pdb_gromacs/topol_Protein_chain_C.itp', '../pdb_gromacs/topol_Protein_chain_D.itp','../pdb_gromacs/topol_Protein_chain_E.itp', '../pdb_gromacs/topol_Protein_chain_F.itp']
lammps_data_file = 'collagen_data_file_no_water'
convert_to_lammps_data(gro_file, itp_files, lammps_data_file)



[('NT', 'O'), ('C', 'C'), ('H', 'CH2'), ('CH2', 'CH2'), ('NT', 'N'), ('H', 'CH1'), ('H', 'C'), ('CH2', 'N'), ('N', 'OM'), ('CH1', 'CH1'), ('CH2', 'OM'), ('CH2', 'C'), ('CH2', 'O'), ('CH1', 'O'), ('H', 'O'), ('N', 'N'), ('CH1', 'CH2'), ('O', 'N')]
[('1', '6'), ('5', '5'), ('2', '4'), ('4', '4'), ('1', '7'), ('2', '3'), ('2', '5'), ('4', '7'), ('7', '8'), ('3', '3'), ('4', '8'), ('4', '5'), ('4', '6'), ('3', '6'), ('2', '6'), ('7', '7'), ('3', '4'), ('6', '7')]


In [7]:
with open(gro_file, 'r') as file:
    lines = file.readlines()
    atom_lines=lines[2:-1]
    x,y,z=[],[],[]
    for line in atom_lines:
        parts = line.split()
        x.append(float(parts[3]))
        y.append(float(parts[4]))
        z.append(float(parts[5]))
print(f'x, y, z max {np.max(x)} {np.max(y)} {np.max(z)}')
print(f'x, y, z min {np.min(x)} {np.min(y)} {np.min(z)}')

x, y, z max 1.715 2.679 18.097
x, y, z min 0.606 1.603 0.113


In [8]:
atom_types = {'NT': 1, 'H': 2, 'CH1': 3, 'CH2':4, 'C':5, 'O':6, 'N':7, 'OM':8}
# pairs = [('7', '7'), ('4', '7'), ('3', '4'), ('4', '6'), ('1', '7'), ('1', '6'), ('4', '5'), 
#          ('4', '4'), ('2', '4'), ('3', '6'), ('3', '3'), ('4', '8'), ('2', '6'), ('2', '5'), 
#          ('5', '5'), ('7', '8'), ('6', '7'), ('2', '3')]

pairs_data = [
    ('N', 'N', 1, 0.0024364096, 1.692601e-06),
    ('CH2', 'N', 1, 0.0033925128, 3.469767e-06),
    ('CH1', 'CH2', 1, 0.0037086708, 5.155311e-06),
    ('CH2', 'O', 1, 0.0032687988, 2.2965537e-06),
    ('NT', 'N', 1, 0.0024364096, 1.692601e-06),
    ('NT', 'O', 1, 0.0023475616, 1.1202911e-06),
    ('CH2', 'C', 1, 0.0033251574, 4.899279e-06),
    ('CH2', 'CH2', 1, 0.0047238129, 7.112889e-06),
    ('H', 'CH2', 1, 0, 0),
    ('CH1', 'O', 1, 0.0025663376, 1.6645063e-06),
    ('CH1', 'CH1', 1, 0.0029116816, 3.736489e-06),
    ('CH2', 'OM', 1, 0.0032687988, 2.2965537e-06),
    ('H', 'O', 1, 0, 0),
    ('H', 'C', 1, 0, 0),
    ('C', 'C', 1, 0.0023406244, 3.374569e-06),
    ('N', 'OM', 1, 0.0023475616, 1.1202911e-06),
    ('O', 'N', 1, 0.0023475616, 1.1202911e-06),
    ('H', 'CH1', 1, 0, 0)
]


# Function to convert c6 and c12 to epsilon and sigma
def convert_c6_c12_to_epsilon_sigma(c6, c12):
    sigma = (c12 / c6) ** (1/6) if c6 != 0 else 0
    epsilon = c6**2 / (4 * sigma**6) if sigma != 0 else 0
    return epsilon, sigma

# Loop through pairs_data and print out the LAMMPS pair_coeff commands
for pair in pairs_data:
    atom_type1, atom_type2, func, c6, c12 = pair
    epsilon, sigma = convert_c6_c12_to_epsilon_sigma(c6, c12)

    if epsilon != 0 and sigma != 0:
        print(f"pair_coeff {atom_types[atom_type1]} {atom_types[atom_type2]} {epsilon} {sigma}")
    else:
        print(f"# pair_coeff {atom_types[atom_type1]} {atom_types[atom_type2]} 0 0")

pair_coeff 7 7 0.002136172508954038 0.29760076860287393
pair_coeff 4 7 0.002813223141319898 0.31741672241291014
pair_coeff 3 4 0.002473659927435566 0.3340715028567879
pair_coeff 4 6 0.0038021388546775267 0.2981591002454447
pair_coeff 1 7 0.002136172508954038 0.29760076860287393
pair_coeff 1 6 0.0028870886128063975 0.2795453772078163
pair_coeff 4 5 0.001876050222811209 0.3373287269619898
pair_coeff 4 4 0.003704862041658299 0.3385521352661637
# pair_coeff 2 4 0 0
pair_coeff 3 6 0.0025386096479726866 0.29421305711486523
pair_coeff 3 3 0.001651611684267103 0.32965017022637283
pair_coeff 4 8 0.0038021388546775267 0.2981591002454447
# pair_coeff 2 6 0 0
# pair_coeff 2 5 0 0
pair_coeff 5 5 0.0009499852893131287 0.33610973962499585
pair_coeff 7 8 0.0028870886128063975 0.2795453772078163
pair_coeff 6 7 0.0028870886128063975 0.2795453772078163
# pair_coeff 2 3 0 0


In [9]:
import itertools

# Define your atom types with their numeric identifiers
atom_types = {'NT': 1, 'H': 2, 'CH1': 3, 'CH2': 4, 'C': 5, 'O': 6, 'N': 7, 'OM': 8}

# Provided pair_coeffs with numeric atom types
provided_pair_coeffs = [
    (7, 7), (4, 7), (3, 4), (4, 6), (1, 7), (1, 6), (4, 5),
    (4, 4), (2, 4), (3, 6), (3, 3), (4, 8), (2, 6), (2, 5),
    (5, 5), (7, 8), (6, 7), (2, 3)
]


# Convert to set for easier comparison
provided_pair_coeffs_set = {tuple(sorted(pair)) for pair in provided_pair_coeffs}

# Generate all possible combinations of atom type numbers
all_pairs = set(itertools.combinations_with_replacement(atom_types.values(), 2))

# Identify missing pair_coeffs
missing_pair_coeffs = all_pairs - provided_pair_coeffs_set

for pair in sorted(missing_pair_coeffs):
    print(f"pair_coeff {pair[0]} {pair[1]} 0.0 0.0")


pair_coeff 1 1 0.0 0.0
pair_coeff 1 2 0.0 0.0
pair_coeff 1 3 0.0 0.0
pair_coeff 1 4 0.0 0.0
pair_coeff 1 5 0.0 0.0
pair_coeff 1 8 0.0 0.0
pair_coeff 2 2 0.0 0.0
pair_coeff 2 7 0.0 0.0
pair_coeff 2 8 0.0 0.0
pair_coeff 3 5 0.0 0.0
pair_coeff 3 7 0.0 0.0
pair_coeff 3 8 0.0 0.0
pair_coeff 5 6 0.0 0.0
pair_coeff 5 7 0.0 0.0
pair_coeff 5 8 0.0 0.0
pair_coeff 6 6 0.0 0.0
pair_coeff 6 8 0.0 0.0
pair_coeff 8 8 0.0 0.0


In [10]:
def convert_to_lammps_data(gro_file, itp_files, lammps_data_file):
    atoms, bonds, angles, dihedrals, impropers, pairs  = [], [], [], [], [], []
    # Process .gro and .itp files
    process_gro_file(gro_file, atoms)
    i=0
    for itp_file in itp_files:
        process_itp_file(itp_file, atoms, bonds, angles, dihedrals, impropers, pairs, i)
        i+=1

    return atoms, bonds, angles, dihedrals

gro_file = '../pdb_gromacs/output.gro'
itp_files = ['../pdb_gromacs/topol_Protein_chain_A.itp', '../pdb_gromacs/topol_Protein_chain_B.itp','../pdb_gromacs/topol_Protein_chain_C.itp', '../pdb_gromacs/topol_Protein_chain_D.itp','../pdb_gromacs/topol_Protein_chain_E.itp', '../pdb_gromacs/topol_Protein_chain_F.itp']

atoms, bonds, angles, dihedrals=convert_to_lammps_data(gro_file,itp_files,None)


In [11]:
# Function to calculate bond length
def calculate_bond_length(atom1, atom2):
    x1, y1, z1 = atom1['x'], atom1['y'], atom1['z']
    x2, y2, z2 = atom2['x'], atom2['y'], atom2['z']
    
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
bonds_length=[]
# Check bond lengths
for bond in bonds:
    atom_id1 = bond['atom_id1']
    atom_id2 = bond['atom_id2']
    
    atom1 = next((atom for atom in atoms if atom['atom_id'] == atom_id1), None)
    atom2 = next((atom for atom in atoms if atom['atom_id'] == atom_id2), None)
    
    if atom1 is not None and atom2 is not None:
        bond_length = calculate_bond_length(atom1, atom2)
        bonds_length.append(bond_length)
        if bond_length > 1.6:
            print(f"Bond ID: {bond['bond_type']}, Atom IDs: {atom_id1}, {atom_id2}, Length: {bond_length:.2f}")

np.sum(bonds_length)

1750.9693420845165